In [1]:
import itertools
import random
import copy

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import utils

utils.font_setting()

In [2]:
# Hyperparameter
sampling_type = 0
sample_N = 50000
seed = 10
n_obs_list = [2, 3, 4, 5, 6]
Vmax_list = [i+0.5 for i in [1, 2, 3, 4, 5]]
theta_FOV_list = [i*np.pi/180 for i in [5, 10, 30, 60]]
theta_img_list = [np.pi/180*10**i for i in [
    -5.0, -4.0, -3.0, -2.75, -2.5, -2.25, -2.0, -1.75, -1.5, -1.25, -1.0]]
k_list = [2.0**i for i in [0.0, 0.5, 1.0]]
# param_col, 'Vmax', 'theta_FOV', 'theta_img', 'k'

# system
log_dir = './log/subgraph_monte/'

# epsilon
epsilon_list_temp = []
for tuple in list(itertools.product(theta_img_list, k_list)):
    eps = tuple[0] * tuple[1]
    epsilon_list_temp.append(eps)
epsilon_list = list(set(epsilon_list_temp))

# Load
df_summary = pd.read_csv(log_dir + f'summary_{sample_N}_{seed}.csv', index_col=0)
df_summary.head()

,n_obs,Vmax,theta_FOV,theta_img,k,epsilon,calc_num,obs_num,time_mean,matching_num_mean,...,notincluded_prob,included_unique_num,included_unique_prob,notincluded_unique_num,notincluded_unique_prob,notincluded_noexist_num,notincluded_noexist_prob,included_on_unique_prob,notincluded_on_unique_prob,noexist_on_notincluded_prob
0,2,1.5,0.087266,1.745329e-07,1.000000,1.745329e-07,50000,136,0.0,0.963235,...,0.036765,131.0,0.963235,0.0,0.0,5.0,0.036765,1.0,0.0,1.0
1,2,1.5,0.087266,1.745329e-07,1.414214,2.468268e-07,50000,136,0.0,1.000000,...,0.000000,136.0,1.000000,0.0,0.0,0.0,0.000000,1.0,0.0,NaN
2,2,1.5,0.087266,1.745329e-07,2.000000,3.490659e-07,50000,136,0.0,1.000000,...,0.000000,136.0,1.000000,0.0,0.0,0.0,0.000000,1.0,0.0,NaN
3,2,1.5,0.087266,1.745329e-06,1.000000,1.745329e-06,50000,136,0.0,0.963235,...,0.036765,131.0,0.963235,0.0,0.0,5.0,0.036765,1.0,0.0,1.0
4,2,1.5,0.087266,1.745329e-06,1.414214,2.468268e-06,50000,136,0.0,1.000000,...,0.000000,136.0,1.000000,0.0,0.0,0.0,0.000000,1.0,0.0,NaN


In [3]:
print(df_summary.columns)

Index(['n_obs', 'Vmax', 'theta_FOV', 'theta_img', 'k', 'epsilon', 'calc_num',
       'obs_num', 'time_mean', 'matching_num_mean', 'obs_prob', 'unique_num',
       'unique_prob', 'ambiguous_num', 'ambiguous_prob', 'multiple_num',
       'multiple_prob', 'noexist_num', 'noexist_prob', 'included_num',
       'included_prob', 'notincluded_num', 'notincluded_prob',
       'included_unique_num', 'included_unique_prob', 'notincluded_unique_num',
       'notincluded_unique_prob', 'notincluded_noexist_num',
       'notincluded_noexist_prob', 'included_on_unique_prob',
       'notincluded_on_unique_prob', 'noexist_on_notincluded_prob'],
      dtype='object')


In [4]:

def split_df(df, condi_col, condi_value_list):
    df_list = []
    condition_params_list = list(itertools.product(*condi_value_list))
    for condition_params in condition_params_list:
        c = True
        for j in range(len(condi_col)):
            c_temp_low = (df[condi_col[j]] > condition_params[j]-np.abs(condition_params[j])*1.0e-1)
            c_temp_high = (df[condi_col[j]] < condition_params[j]+np.abs(condition_params[j])*1.0e-1)
            c = c & c_temp_low & c_temp_high
        df_list.append(df[c])
    return condition_params_list, df_list


def plot_tar(df_summary, target_col, condi_col, condi_value_list):
    scale = 2
    fig = plt.figure(figsize=(scale*1.6*len(condi_col), scale))
    axes = []
    for i in range(len(condi_col)):
        axes.append(fig.add_subplot(1, len(condi_col), i+1))
    
    for i, param_col in enumerate(condi_col):
        condi_cols = copy.copy(condi_col)
        condi_cols.pop(i)
        condi_value_lists = copy.copy(condi_value_list)
        condi_value_lists.pop(i)
        # 
        condition_params_list, df_list = split_df(df_summary, condi_cols, condi_value_lists)
        for df in df_list:
            axes[i].plot(df[param_col], df[target_col], ls='-', lw=0.2)
            axes[i].set_ylabel(target_col)
            axes[i].set_xlabel(param_col)
    # 
    fig.tight_layout()
    fig.savefig(f'./img/{target_col}.pdf')
    plt.close(fig)


In [5]:
# df_summary = df_summary[df_summary['theta_FOV'] == 30*np.pi/180]
condi_col = ['n_obs', 'Vmax', 'theta_FOV', 'theta_img', 'k']
condi_value_list = [n_obs_list, Vmax_list, theta_FOV_list, theta_img_list, k_list]

plot_tar(df_summary, 'time_mean', condi_col, condi_value_list)
plot_tar(df_summary, 'matching_num_mean', condi_col, condi_value_list)
plot_tar(df_summary, 'obs_prob', condi_col, condi_value_list)
plot_tar(df_summary, 'unique_prob', condi_col, condi_value_list)
plot_tar(df_summary, 'ambiguous_prob', condi_col, condi_value_list)
plot_tar(df_summary, 'multiple_prob', condi_col, condi_value_list)
plot_tar(df_summary, 'noexist_prob', condi_col, condi_value_list)
plot_tar(df_summary, 'included_prob', condi_col, condi_value_list)
plot_tar(df_summary, 'notincluded_prob', condi_col, condi_value_list)
plot_tar(df_summary, 'included_unique_prob', condi_col, condi_value_list)
plot_tar(df_summary, 'notincluded_unique_prob', condi_col, condi_value_list)
plot_tar(df_summary, 'notincluded_noexist_prob', condi_col, condi_value_list)
plot_tar(df_summary, 'included_on_unique_prob', condi_col, condi_value_list)
plot_tar(df_summary, 'notincluded_on_unique_prob', condi_col, condi_value_list)
plot_tar(df_summary, 'noexist_on_notincluded_prob', condi_col, condi_value_list)
